### Load Dataset from Kaggle

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("olgabelitskaya/horse-breeds")

print("Path to dataset files:", path)

100%|██████████| 82.0M/82.0M [00:00<00:00, 172MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/olgabelitskaya/horse-breeds/versions/27


### installs and imports

In [2]:
pip install numpy torch torchvision matplotlib -qqq

In [4]:
!pip install torch torchvision numpy matplotlib


In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
